# SQLAlchemy Core 튜토리얼

ORM은 domain model의 관점에서 데이터에 접근하지만, 표현식 언어(Core 방식)는 데이터베이스에서 사용되는 쿼리를 만들어준다.

ORM을 사용해도 종종 특정 데이터베이스와 상호작용이 필요하면 표현식을 사용한다.

![SQLAlchemy](http://docs.sqlalchemy.org/en/latest/_images/sqla_arch_small.png)

In [1]:
from sqlalchemy import *

## 데이터베이스 연결

- `create_engine()`로 연결
- 인수로 들어가는 URL: dialect[+driver]://user:password@host/dbname[?key=value..]

In [2]:
engine = create_engine('sqlite:///:memory:', echo=True)

## 테이블 정의하고 생성하기

- 데이터베이스 메타데이터(MetaData) : Table 객체와 연결된 자식 객체의 집합
- `Table` 구문으로 테이블을 정의한다.

In [3]:
# 테이블 모양 정의
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey
metadata = MetaData()
users = Table('users', metadata,
             Column('id', Integer, primary_key=True),
             Column('name', String),
             Column('fullname', String),
             )

addresses = Table('addresses', metadata,
             Column('id', Integer, primary_key=True),
             Column('user_id', None, ForeignKey('users.id')),
             Column('email_address', String, nullable=False),
             )

In [4]:
# create_all 을 통해서 metadata에 있는 정보를 engine에 전달하고 테이블 생성
metadata.create_all(engine)

2018-06-07 09:59:09,619 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-06-07 09:59:09,620 INFO sqlalchemy.engine.base.Engine ()
2018-06-07 09:59:09,623 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-06-07 09:59:09,624 INFO sqlalchemy.engine.base.Engine ()
2018-06-07 09:59:09,625 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2018-06-07 09:59:09,626 INFO sqlalchemy.engine.base.Engine ()
2018-06-07 09:59:09,628 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("addresses")
2018-06-07 09:59:09,629 INFO sqlalchemy.engine.base.Engine ()
2018-06-07 09:59:09,630 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2018-06-07 09:59:09,631 INFO sqlalchemy.engine.base.Engine ()
2018-06-07 09:59:09,632 INFO sqlalchemy.engine.base.Engine COMMIT
2018-06-07 09:59:09,633 INFO sqlalchemy.engine.b

## INSERT 표현식
- Insert Expressions
- values에 컬럼 값을 지정해줘도, str()을 통해서 보면 보이지 않고, 실행되어야 보인다.  
- 생성된 바인드 파라미터의 값은 `comile().params`를 통해 확인할 수 있다.
- orm과 마찬가지로 실행하기 전까지는 데이터베이스에 반영되지 않는다.
- 참고
    - ORM에서는 `session`에 `add`를 하고, `commit`을 하면 행이 삽입됐다.

In [5]:
ins = users.insert()
str(ins)

'INSERT INTO users (id, name, fullname) VALUES (:id, :name, :fullname)'

In [6]:
ins = users.insert().values(name='jack', fullname='Jack Jones')
str(ins)

'INSERT INTO users (name, fullname) VALUES (:name, :fullname)'

## 실행

- `execute()`
- 실제 반영되는건 engine object에 연결한 후 실행할 때다.
- engine 객체는 SQL을 DB에 보내기 위한 연결을 위한 장소. 연결을 위해 connect() 메서드 사용.
- 아래에서 conn 객체는 DBAPI 연결 리소스를 나타낸다.

In [7]:
# engine에 연결
conn = engine.connect()
conn

In [8]:
# 데이터베이스에서 실행.
result = conn.execute(ins)

2018-06-07 09:59:16,631 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname) VALUES (?, ?)
2018-06-07 09:59:16,632 INFO sqlalchemy.engine.base.Engine ('jack', 'Jack Jones')
2018-06-07 09:59:16,633 INFO sqlalchemy.engine.base.Engine COMMIT


- `str()`으로 ins의 값을 확인할 때 '?'가 나오는 것은 명령문이 dialect를 모르고 명명된 파라미터를 사용하는 기본값으로 돌아가기 때문이다.

In [9]:
ins.bind = engine
str(ins)

'INSERT INTO users (name, fullname) VALUES (?, ?)'

In [10]:
# result는 DBAPI 커서 객체와 유사하다.
result

- primary_key를 명시적으로 지정하지 않았지만, 자동으로 생성됐다. 명시적으로 지정해줬으면 지정한 값이 들어간다.

In [11]:
result.inserted_primary_key

[1]

In [12]:
# 일반적인 insert() 구문의 사용
ins = users.insert()
conn.execute(ins, id=2, name='wendy', fullname='Wendy Williams')

2018-06-07 09:59:20,481 INFO sqlalchemy.engine.base.Engine INSERT INTO users (id, name, fullname) VALUES (?, ?, ?)
2018-06-07 09:59:20,482 INFO sqlalchemy.engine.base.Engine (2, 'wendy', 'Wendy Williams')
2018-06-07 09:59:20,483 INFO sqlalchemy.engine.base.Engine COMMIT


## 여러개의 명령문 실행
- 딕셔너리의 리스트를 전달하면 여러 데이터를 한꺼번에 insert할 수 있다.
- 딕셔너리로 리스트를 만들 때, 첫번째 딕셔너리의 키 값이 중요하다.
    - 첫번째 딕셔너리의 키 값을 기준으로 명령문이 컴파일됨.
    - 예를 들어, 첫번째 딕셔너리에 id, name을 인수로 사용하면, 그 이후도 모두 동일한 인수를 가져야 함.

In [13]:
conn.execute(addresses.insert(), [
    {'user_id': 1, 'email_address' : 'jack@yahoo.com'},
    {'user_id': 1, 'email_address' : 'jack@msn.com'},
    {'user_id': 2, 'email_address' : 'www@www.org'},
    {'user_id': 2, 'email_address' : 'wendy@aol.com'},
 ])

2018-06-07 09:59:22,331 INFO sqlalchemy.engine.base.Engine INSERT INTO addresses (user_id, email_address) VALUES (?, ?)
2018-06-07 09:59:22,333 INFO sqlalchemy.engine.base.Engine ((1, 'jack@yahoo.com'), (1, 'jack@msn.com'), (2, 'www@www.org'), (2, 'wendy@aol.com'))
2018-06-07 09:59:22,333 INFO sqlalchemy.engine.base.Engine COMMIT


## Selecting

- `select()` 함수 사용

In [14]:
from sqlalchemy.sql import select
s = select([users])
result = conn.execute(s)

2018-06-07 09:59:23,795 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users
2018-06-07 09:59:23,796 INFO sqlalchemy.engine.base.Engine ()


- select를 실행시키면 row가 튜플로 나옴.
- 결과는 사용 후에 명확하게 닫아주는 것이 좋다.

In [15]:
for row in result:
    print(row)
result.close()

(1, 'jack', 'Jack Jones')
(2, 'wendy', 'Wendy Williams')


- `fetchone()`이나 `fetchall()`을 사용하면 딕셔너리로 가져올 수 있음.
    - 문자열 이름을 이용한 딕셔너리 접근
    - 정수 인덱스 사용
    - 가장 좋은 방법 : column 객체를 직접 키로 사용하는 것(`테이블명.c.컬럼명`)

In [16]:
row = conn.execute(s).fetchone()
# 컬럼 이름 사용
print("name:", row['name'], "; fullname:", row['fullname'])
# 인덱스 사용
print("name:", row[1], "; fullname:", row[2])

2018-06-07 09:59:26,282 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users
2018-06-07 09:59:26,283 INFO sqlalchemy.engine.base.Engine ()
name: jack ; fullname: Jack Jones
name: jack ; fullname: Jack Jones


In [17]:
# 컬럼을 키로 사용
for row in conn.execute(s):
    print("name:", row[users.c.name], "; fullname:", row[users.c.fullname])

2018-06-07 09:59:26,707 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users
2018-06-07 09:59:26,707 INFO sqlalchemy.engine.base.Engine ()
name: jack ; fullname: Jack Jones
name: wendy ; fullname: Wendy Williams


- 컬럼을 지정할 수 있다.

In [18]:
# id를 제외한 name, fullname만 선택
s = select([users.c.name, users.c.fullname])
conn.execute(s).fetchall()

2018-06-07 09:59:28,768 INFO sqlalchemy.engine.base.Engine SELECT users.name, users.fullname 
FROM users
2018-06-07 09:59:28,769 INFO sqlalchemy.engine.base.Engine ()


[('jack', 'Jack Jones'), ('wendy', 'Wendy Williams')]

- 두 테이블을 한꺼번에 select하면 Cartesian Product가 나온다.

In [19]:
s = select([users, addresses])
conn.execute(s).fetchall()

2018-06-07 09:59:30,131 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname, addresses.id, addresses.user_id, addresses.email_address 
FROM users, addresses
2018-06-07 09:59:30,132 INFO sqlalchemy.engine.base.Engine ()


[(1, 'jack', 'Jack Jones', 1, 1, 'jack@yahoo.com'),
 (1, 'jack', 'Jack Jones', 2, 1, 'jack@msn.com'),
 (1, 'jack', 'Jack Jones', 3, 2, 'www@www.org'),
 (1, 'jack', 'Jack Jones', 4, 2, 'wendy@aol.com'),
 (2, 'wendy', 'Wendy Williams', 1, 1, 'jack@yahoo.com'),
 (2, 'wendy', 'Wendy Williams', 2, 1, 'jack@msn.com'),
 (2, 'wendy', 'Wendy Williams', 3, 2, 'www@www.org'),
 (2, 'wendy', 'Wendy Williams', 4, 2, 'wendy@aol.com')]

- `where(users.c.id == addresses.c.user_id)`를 추가하면 foriegn key가 적용된 행만 선택된다.
- `==`의 사용에 대해서는 아래에서 더 자세히 살펴본다.

In [20]:
s = select([users, addresses]).where(users.c.id == addresses.c.user_id)
conn.execute(s).fetchall()

2018-06-07 09:59:32,031 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname, addresses.id, addresses.user_id, addresses.email_address 
FROM users, addresses 
WHERE users.id = addresses.user_id
2018-06-07 09:59:32,032 INFO sqlalchemy.engine.base.Engine ()


[(1, 'jack', 'Jack Jones', 1, 1, 'jack@yahoo.com'),
 (1, 'jack', 'Jack Jones', 2, 1, 'jack@msn.com'),
 (2, 'wendy', 'Wendy Williams', 3, 2, 'www@www.org'),
 (2, 'wendy', 'Wendy Williams', 4, 2, 'wendy@aol.com')]

## 연산자
- Operators
    - Operator Customization

- ColumnElement 클래스
- `==`으로 2개 컬럼을 비교하면 `True`, `False`가 아니라 SQL이 생성된다.

In [21]:
print(users.c.id == addresses.c.user_id)

users.id = addresses.user_id


- 리터럴 값을 사용하면, 바인드 파라미터를 얻는다.

In [22]:
# 7은 ColumnElement 결과에 포함됨.
print(users.c.id == 7)
(users.c.id == 7).compile().params

users.id = :id_1


{'id_1': 7}

In [23]:
# 같지 않다.
print(users.c.id != 7)
# 같다.
print(users.c.name == None)
# 크다/작다.
print('fred' > users.c.name)

users.id != :id_1
users.name IS NULL
users.name < :name_1


In [24]:
# 더하기(+)는 데이터 타입에 따라 표현이 달라진다.
# 숫자 : 더하기
print(users.c.id + addresses.c.id)
# 문자 : 문자열 연결 연산자인 " || "
print(users.c.name + users.c.fullname)

users.id + addresses.id
users.name || users.fullname


- 사용할 수 없는 연산자는 Operators.op 메서드를 이용해서 만들 수 있다.
- 불린 자료형 연산자는 `Operators.bool_op` 메서드 사용
- 연산자 사용자 지정도 제공.

In [25]:
print(users.c.name.op('++++')('foo'))

users.name ++++ :name_1


## Conjunction

- AND : .and_, &
- OR : .or_, |
- NOT : .not_, ~

In [26]:
from sqlalchemy.sql import and_, or_, not_

In [27]:
print(and_(
        users.c.name.like('j%'),
        users.c.id == addresses.c.user_id,
        or_(
             addresses.c.email_address == 'wendy@aol.com',
             addresses.c.email_address == 'jack@yahoo.com'
        ),
        not_(users.c.id > 5)
      )
 )

users.name LIKE :name_1 AND users.id = addresses.user_id AND (addresses.email_address = :email_address_1 OR addresses.email_address = :email_address_2) AND users.id <= :id_1


In [28]:
print(users.c.name.like('j%') & (users.c.id == addresses.c.user_id) &
    (
      (addresses.c.email_address == 'wendy@aol.com') | \
      (addresses.c.email_address == 'jack@yahoo.com')
    ) \
    & ~(users.c.id>5)
)

users.name LIKE :name_1 AND users.id = addresses.user_id AND (addresses.email_address = :email_address_1 OR addresses.email_address = :email_address_2) AND users.id <= :id_1


## SQL 문자열 직접 사용하기

- text() 함수를 사용해서 SQL 문자열을 직접 사용할 수 있다.
- 바인드 파라미터를 사용할 수 있다. 파라미터의 타입을 지정할 수도 있다.
- 결과 컬럼 지정
- 명령문 안에서 `text()` 사용

In [29]:
from sqlalchemy.sql import text
s = text(
    "SELECT users.fullname || ', ' || addresses.email_address AS title "
        "FROM users, addresses "
        "WHERE users.id = addresses.user_id "
        "AND users.name BETWEEN :x AND :y "
        "AND (addresses.email_address LIKE :e1 "
            "OR addresses.email_address LIKE :e2)")
conn.execute(s, x='m', y='z', e1='%@aol.com', e2='%@msn.com').fetchall()

2018-06-07 09:59:42,972 INFO sqlalchemy.engine.base.Engine SELECT users.fullname || ', ' || addresses.email_address AS title FROM users, addresses WHERE users.id = addresses.user_id AND users.name BETWEEN ? AND ? AND (addresses.email_address LIKE ? OR addresses.email_address LIKE ?)
2018-06-07 09:59:42,973 INFO sqlalchemy.engine.base.Engine ('m', 'z', '%@aol.com', '%@msn.com')


[('Wendy Williams, wendy@aol.com',)]

In [30]:
# 바인드 파라미터
stmt = text("SELECT * FROM users WHERE users.name BETWEEN :x AND :y")
stmt = stmt.bindparams(x="m", y="z")

In [31]:
# 바인드 파라미터의 데이터 타입 지정
stmt = stmt.bindparams(bindparam("x", String), bindparam("y", String))
conn.execute(stmt, {"x": "m", "y": "z"})

2018-06-07 09:59:45,070 INFO sqlalchemy.engine.base.Engine SELECT * FROM users WHERE users.name BETWEEN ? AND ?
2018-06-07 09:59:45,072 INFO sqlalchemy.engine.base.Engine ('m', 'z')


In [32]:
stmt = text("SELECT users.id, addresses.id, users.id, "
    "users.name, addresses.email_address AS email "
    "FROM users JOIN addresses ON users.id=addresses.user_id "
    "WHERE users.id = 1").columns(
       users.c.id,
       addresses.c.id,
       addresses.c.user_id,
       users.c.name,
       addresses.c.email_address
    )
result = conn.execute(stmt)

2018-06-07 09:59:45,548 INFO sqlalchemy.engine.base.Engine SELECT users.id, addresses.id, users.id, users.name, addresses.email_address AS email FROM users JOIN addresses ON users.id=addresses.user_id WHERE users.id = 1
2018-06-07 09:59:45,549 INFO sqlalchemy.engine.base.Engine ()


In [33]:
row = result.fetchone()
row[addresses.c.email_address]

'jack@yahoo.com'

In [34]:
# 일부만 text() 사용
s = select([users, addresses]).where(text("users.id = addresses.user_id"))
conn.execute(s).fetchall()

2018-06-07 09:59:47,519 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname, addresses.id, addresses.user_id, addresses.email_address 
FROM users, addresses 
WHERE users.id = addresses.user_id
2018-06-07 09:59:47,520 INFO sqlalchemy.engine.base.Engine ()


[(1, 'jack', 'Jack Jones', 1, 1, 'jack@yahoo.com'),
 (1, 'jack', 'Jack Jones', 2, 1, 'jack@msn.com'),
 (2, 'wendy', 'Wendy Williams', 3, 2, 'www@www.org'),
 (2, 'wendy', 'Wendy Williams', 4, 2, 'wendy@aol.com')]

### 앨리어스(Alias) 사용하기
- AS 절과 같은 역할
- `alias()`

In [35]:
a1 = addresses.alias('a1')
a2 = addresses.alias()
# 참고 - orm에서의 alias
# adalias1 = aliased(Address)
s = select([users]).\
        where(and_(
            users.c.id == a1.c.user_id, # 이름이 'a1'으로 지정됨. 
            users.c.id == a2.c.user_id, # 이름을 지정하지 않아서 'addresses_1'으로 자동생성.
            a1.c.email_address == 'jack@msn.com',
            a2.c.email_address == 'jack@yahoo.com'
        ))
conn.execute(s).fetchall()

2018-06-07 09:59:49,384 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users, addresses AS a1, addresses AS addresses_1 
WHERE users.id = a1.user_id AND users.id = addresses_1.user_id AND a1.email_address = ? AND addresses_1.email_address = ?
2018-06-07 09:59:49,385 INFO sqlalchemy.engine.base.Engine ('jack@msn.com', 'jack@yahoo.com')


[(1, 'jack', 'Jack Jones')]

## JOIN 사용하기
- `select` 안에서 사용 시 `select_from` 메서드를 사용한다.
    - `join()` : JOIN
    - `outerjoin()` : LEFT OUTER JOIN

In [36]:
print(users.join(addresses))

users JOIN addresses ON users.id = addresses.user_id


In [37]:
# select 안에서 사용하면 select_from 메서드를 사용한다.
s = select([users.c.fullname, addresses.c.email_address]).select_from(
   users.join(addresses)
   )
conn.execute(s).fetchall()

2018-06-07 09:59:51,545 INFO sqlalchemy.engine.base.Engine SELECT users.fullname, addresses.email_address 
FROM users JOIN addresses ON users.id = addresses.user_id
2018-06-07 09:59:51,546 INFO sqlalchemy.engine.base.Engine ()


[('Jack Jones', 'jack@yahoo.com'),
 ('Jack Jones', 'jack@msn.com'),
 ('Wendy Williams', 'www@www.org'),
 ('Wendy Williams', 'wendy@aol.com')]

- outerjoin 메서드를 사용하면 LEFT OUTER JOIN 생성

In [38]:
print(users.outerjoin(addresses))

users LEFT OUTER JOIN addresses ON users.id = addresses.user_id


In [39]:
s = select([users.c.fullname, addresses.c.email_address]).select_from(users.outerjoin(addresses))
conn.execute(s).fetchall()

2018-06-07 09:59:52,894 INFO sqlalchemy.engine.base.Engine SELECT users.fullname, addresses.email_address 
FROM users LEFT OUTER JOIN addresses ON users.id = addresses.user_id
2018-06-07 09:59:52,895 INFO sqlalchemy.engine.base.Engine ()


[('Jack Jones', 'jack@msn.com'),
 ('Jack Jones', 'jack@yahoo.com'),
 ('Wendy Williams', 'wendy@aol.com'),
 ('Wendy Williams', 'www@www.org')]

## 바인드 파라미터 객체

- `bindparam()`을 통해 바인드파라미터를 생성할 수 있다.
- `type_`을 인수로 넣어서 데이터타입을 지정할 수 있다.

In [40]:
from sqlalchemy.sql import bindparam
s = users.select(users.c.name == bindparam('username', type_=String))
conn.execute(s, username='wendy').fetchall()

2018-06-07 09:59:54,772 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users 
WHERE users.name = ?
2018-06-07 09:59:54,773 INFO sqlalchemy.engine.base.Engine ('wendy',)


[(2, 'wendy', 'Wendy Williams')]

## 함수

- 함수는 `func` 키워드를 통해 사용한다.
- 일반적으로 SELECT 문의 컬럼 절에서 사용한다.
- 윈도우 함수 : OVER 절
    - `FunctionElement.over()` 메서드로 사용한다.

In [41]:
from sqlalchemy.sql import func

In [42]:
print(func.count())

count(*)


In [43]:
# label은 결과 컬럼에 이름 생성. label을 붙이지 않으면 자동으로 컬럼이름이 생성됨.
# scalar()를 이용해 첫 행의 첫 열을 읽은 후 결과를 닫음.
conn.execute(
    select([
           func.max(addresses.c.email_address, type_=String).label('maxemail')
          ])
    ).scalar()

2018-06-07 09:59:58,674 INFO sqlalchemy.engine.base.Engine SELECT max(addresses.email_address) AS maxemail 
FROM addresses
2018-06-07 09:59:58,675 INFO sqlalchemy.engine.base.Engine ()


'www@www.org'

In [44]:
s = select([
        users.c.id,
        func.row_number().over(order_by=users.c.name)
    ])
print(s)

SELECT users.id, row_number() OVER (ORDER BY users.name) AS anon_1 
FROM users


## 집합 연산

- 합집합: `union()`, `union_all()`
- 교집합: `intersect()`, `intersect_all()`
- 차집합: `except_()`, `except_all()`
- `all`이 붙어있는 함수는 중복을 제거하지 않음.

In [45]:
from sqlalchemy.sql import union, union_all, except_, except_all

u = union_all(
    addresses.select().
            where(addresses.c.email_address == 'www@www.org'),
    addresses.select().
            where(addresses.c.email_address.like('%@yahoo.com')),
).order_by(addresses.c.email_address)

conn.execute(u).fetchall()

2018-06-07 10:00:00,735 INFO sqlalchemy.engine.base.Engine SELECT addresses.id, addresses.user_id, addresses.email_address 
FROM addresses 
WHERE addresses.email_address = ? UNION ALL SELECT addresses.id, addresses.user_id, addresses.email_address 
FROM addresses 
WHERE addresses.email_address LIKE ? ORDER BY addresses.email_address
2018-06-07 10:00:00,736 INFO sqlalchemy.engine.base.Engine ('www@www.org', '%@yahoo.com')


[(1, 1, 'jack@yahoo.com'), (3, 2, 'www@www.org')]

In [46]:
from sqlalchemy.sql import except_
u = except_(
    addresses.select().
             where(addresses.c.email_address.like('%@%.com')),
    addresses.select().
             where(addresses.c.email_address.like('%@msn.com'))
 )

conn.execute(u).fetchall()

2018-06-07 10:00:01,375 INFO sqlalchemy.engine.base.Engine SELECT addresses.id, addresses.user_id, addresses.email_address 
FROM addresses 
WHERE addresses.email_address LIKE ? EXCEPT SELECT addresses.id, addresses.user_id, addresses.email_address 
FROM addresses 
WHERE addresses.email_address LIKE ?
2018-06-07 10:00:01,376 INFO sqlalchemy.engine.base.Engine ('%@%.com', '%@msn.com')


[(1, 1, 'jack@yahoo.com'), (4, 2, 'wendy@aol.com')]

## Scalar Selects
- scalar select는 첫 행의 첫번째 컬럼을 반환한다.
- `label()`을 이용해서 이름을 적용할 수 있다.

In [47]:
stmt = select([func.count(addresses.c.id)]).\
            where(users.c.id == addresses.c.user_id).\
            as_scalar()
conn.execute(select([users.c.name, stmt])).fetchall()

2018-06-07 10:00:03,158 INFO sqlalchemy.engine.base.Engine SELECT users.name, (SELECT count(addresses.id) AS count_1 
FROM addresses 
WHERE users.id = addresses.user_id) AS anon_1 
FROM users
2018-06-07 10:00:03,159 INFO sqlalchemy.engine.base.Engine ()


[('jack', 2), ('wendy', 2)]

## 상호연관 서브쿼리(Correlated Subquery)

- 메인쿼리의 값을 서브쿼리가 사용하고, 서브쿼리가 수행한 결과를 다시 메인쿼리가 받아서 계산하는 구조
- `correalate()` 메서드
- LATERAL 상관관계 : `lateral()` 메서드

In [48]:
stmt = select([addresses.c.user_id]).\
            where(addresses.c.user_id == users.c.id).\
            where(addresses.c.email_address == 'jack@yahoo.com')
enclosing_stmt = select([users.c.name]).where(users.c.id == stmt)
conn.execute(enclosing_stmt).fetchall()

2018-06-07 10:00:04,020 INFO sqlalchemy.engine.base.Engine SELECT users.name 
FROM users 
WHERE users.id = (SELECT addresses.user_id 
FROM addresses 
WHERE addresses.user_id = users.id AND addresses.email_address = ?)
2018-06-07 10:00:04,021 INFO sqlalchemy.engine.base.Engine ('jack@yahoo.com',)


[('jack',)]

In [49]:
# correalate() 메서드
stmt = select([users.c.id]).\
            where(users.c.id == addresses.c.user_id).\
            where(users.c.name == 'jack').\
            correlate(addresses)
enclosing_stmt = select(
        [users.c.name, addresses.c.email_address]).\
    select_from(users.join(addresses)).\
    where(users.c.id == stmt)
conn.execute(enclosing_stmt).fetchall()

2018-06-07 10:00:05,798 INFO sqlalchemy.engine.base.Engine SELECT users.name, addresses.email_address 
FROM users JOIN addresses ON users.id = addresses.user_id 
WHERE users.id = (SELECT users.id 
FROM users 
WHERE users.id = addresses.user_id AND users.name = ?)
2018-06-07 10:00:05,799 INFO sqlalchemy.engine.base.Engine ('jack',)


[('jack', 'jack@yahoo.com'), ('jack', 'jack@msn.com')]

- LATERAL 상관관계
    - `table1 JOIN LATERAL (some SELECT) AS subquery`와 같은 쿼리
    - join을 lateral과 함께 사용하면, 서브쿼리 안에 없는 테이블까지 참조할 수 있다. 그래서 일종의 상관관계라고 할 수 있다.
    - 아래 예에서 JOIN에 LATERAL을 사용했기 때문에 서브쿼리의 SELECT가 서브쿼리 밖에 있는 people 테이블을 참조할 수 있다.
    ```
    SELECT people.people_id, people.age, people.name 
    FROM people JOIN LATERAL (SELECT books.book_id AS book_id 
    FROM books 
    WHERE books.owner_id = people.people_id) AS book_subq ON true
    ```
    - LATERAL은 원래 JOIN과 함께만 사용되는 것은 아니다. 서브쿼리와 함께 사용된다.
    
    - 참고 링크
        1. [LATERAL join](https://heapanalytics.com/blog/engineering/postgresqls-powerful-new-join-type-lateral)
        2. [LATERAL Subqueries](https://www.postgresql.org/docs/current/static/queries-table-expressions.html#QUERIES-LATERAL)
        3. [stack overflow: difference between LATERAL and a subquery in PostgreSQL](https://stackoverflow.com/questions/28550679/what-is-the-difference-between-lateral-and-a-subquery-in-postgresql)

In [50]:
from sqlalchemy import table, column, select, true
people = table('people', column('people_id'), column('age'), column('name'))
books = table('books', column('book_id'), column('owner_id'))
subq = select([books.c.book_id]).\
     where(books.c.owner_id == people.c.people_id).lateral("book_subq")
print(select([people]).select_from(people.join(subq, true())))

SELECT people.people_id, people.age, people.name 
FROM people JOIN LATERAL (SELECT books.book_id AS book_id 
FROM books 
WHERE books.owner_id = people.people_id) AS book_subq ON true


## Ordering, Grouping, Limiting, Offset

- 정렬(ORDER BY) : `order_by()`
    - 오름차순/내림차순 : `asc()`, `desc()` 수정자
- 그룹화(GROUP BY) : `group_by()`
    - HAVING : `having()`
- 중복제거(DISTINCT) : `distinct()`
- LIMIT : `limit()`
- OFFSET : `offset()`

In [51]:
# ORDER BY
stmt = select([users.c.name]).order_by(users.c.name.desc())
conn.execute(stmt).fetchall()

2018-06-07 10:00:12,382 INFO sqlalchemy.engine.base.Engine SELECT users.name 
FROM users ORDER BY users.name DESC
2018-06-07 10:00:12,383 INFO sqlalchemy.engine.base.Engine ()


[('wendy',), ('jack',)]

In [52]:
# GROUP BY
stmt = select([users.c.name, func.count(addresses.c.id)]).\
            select_from(users.join(addresses)).\
            group_by(users.c.name)
conn.execute(stmt).fetchall()

2018-06-07 10:00:12,971 INFO sqlalchemy.engine.base.Engine SELECT users.name, count(addresses.id) AS count_1 
FROM users JOIN addresses ON users.id = addresses.user_id GROUP BY users.name
2018-06-07 10:00:12,972 INFO sqlalchemy.engine.base.Engine ()


[('jack', 2), ('wendy', 2)]

In [53]:
# GROUP BY에 HAVING 추가
stmt = select([users.c.name, func.count(addresses.c.id)]).\
            select_from(users.join(addresses)).\
            group_by(users.c.name).\
            having(func.length(users.c.name) > 4)
conn.execute(stmt).fetchall()

2018-06-07 10:00:14,509 INFO sqlalchemy.engine.base.Engine SELECT users.name, count(addresses.id) AS count_1 
FROM users JOIN addresses ON users.id = addresses.user_id GROUP BY users.name 
HAVING length(users.name) > ?
2018-06-07 10:00:14,510 INFO sqlalchemy.engine.base.Engine (4,)


[('wendy', 2)]

In [54]:
# SELECT 문에서 사용되는 DISTINCT 수정자
stmt = select([users.c.name]).\
            where(addresses.c.email_address.
                   contains(users.c.name)).\
            distinct()
conn.execute(stmt).fetchall()

2018-06-07 10:00:15,021 INFO sqlalchemy.engine.base.Engine SELECT DISTINCT users.name 
FROM users, addresses 
WHERE (addresses.email_address LIKE '%' || users.name || '%')
2018-06-07 10:00:15,023 INFO sqlalchemy.engine.base.Engine ()


[('jack',), ('wendy',)]

In [55]:
# LIMIT, OFFSET
stmt = select([users.c.name, addresses.c.email_address]).\
            select_from(users.join(addresses)).\
            limit(1).offset(1)
conn.execute(stmt).fetchall()

2018-06-07 10:00:15,687 INFO sqlalchemy.engine.base.Engine SELECT users.name, addresses.email_address 
FROM users JOIN addresses ON users.id = addresses.user_id
 LIMIT ? OFFSET ?
2018-06-07 10:00:15,688 INFO sqlalchemy.engine.base.Engine (1, 1)


[('jack', 'jack@msn.com')]

## Updates

- UPDATE: `update()`

In [56]:
stmt = users.update().\
            values(fullname="Fullname: " + users.c.name)
conn.execute(stmt)
conn.execute(users.select()).fetchall()

2018-06-07 10:00:17,070 INFO sqlalchemy.engine.base.Engine UPDATE users SET fullname=(? || users.name)
2018-06-07 10:00:17,071 INFO sqlalchemy.engine.base.Engine ('Fullname: ',)
2018-06-07 10:00:17,071 INFO sqlalchemy.engine.base.Engine COMMIT
2018-06-07 10:00:17,073 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users
2018-06-07 10:00:17,074 INFO sqlalchemy.engine.base.Engine ()


[(1, 'jack', 'Fullname: jack'), (2, 'wendy', 'Fullname: wendy')]

In [57]:
# bindparam()을 사용해서 여러 행 입력
stmt = users.insert().\
        values(name=bindparam('_name') + " .. name")
conn.execute(stmt, [
       {'id':4, '_name':'name1'},
       {'id':5, '_name':'name2'},
       {'id':6, '_name':'name3'},
    ])
conn.execute(users.select()).fetchall()

2018-06-07 10:00:19,458 INFO sqlalchemy.engine.base.Engine INSERT INTO users (id, name) VALUES (?, (? || ?))
2018-06-07 10:00:19,459 INFO sqlalchemy.engine.base.Engine ((4, 'name1', ' .. name'), (5, 'name2', ' .. name'), (6, 'name3', ' .. name'))
2018-06-07 10:00:19,460 INFO sqlalchemy.engine.base.Engine COMMIT
2018-06-07 10:00:19,462 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users
2018-06-07 10:00:19,462 INFO sqlalchemy.engine.base.Engine ()


[(1, 'jack', 'Fullname: jack'),
 (2, 'wendy', 'Fullname: wendy'),
 (4, 'name1 .. name', None),
 (5, 'name2 .. name', None),
 (6, 'name3 .. name', None)]

In [58]:
stmt = users.update().\
            where(users.c.name == 'jack').\
            values(name='ed')
conn.execute(stmt)
conn.execute(users.select()).fetchall()

2018-06-07 10:00:20,407 INFO sqlalchemy.engine.base.Engine UPDATE users SET name=? WHERE users.name = ?
2018-06-07 10:00:20,408 INFO sqlalchemy.engine.base.Engine ('ed', 'jack')
2018-06-07 10:00:20,409 INFO sqlalchemy.engine.base.Engine COMMIT
2018-06-07 10:00:20,411 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users
2018-06-07 10:00:20,412 INFO sqlalchemy.engine.base.Engine ()


[(1, 'ed', 'Fullname: jack'),
 (2, 'wendy', 'Fullname: wendy'),
 (4, 'name1 .. name', None),
 (5, 'name2 .. name', None),
 (6, 'name3 .. name', None)]

In [59]:
stmt = users.update().\
            where(users.c.name == bindparam('oldname')).\
            values(name=bindparam('newname'))
conn.execute(stmt, [
    {'oldname':'jack', 'newname':'ed'},
    {'oldname':'wendy', 'newname':'mary'},
    ])

2018-06-07 10:00:21,446 INFO sqlalchemy.engine.base.Engine UPDATE users SET name=? WHERE users.name = ?
2018-06-07 10:00:21,447 INFO sqlalchemy.engine.base.Engine (('ed', 'jack'), ('mary', 'wendy'))
2018-06-07 10:00:21,449 INFO sqlalchemy.engine.base.Engine COMMIT


In [60]:
conn.execute(users.select()).fetchall()

2018-06-07 10:00:23,657 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users
2018-06-07 10:00:23,658 INFO sqlalchemy.engine.base.Engine ()


[(1, 'ed', 'Fullname: jack'),
 (2, 'mary', 'Fullname: wendy'),
 (4, 'name1 .. name', None),
 (5, 'name2 .. name', None),
 (6, 'name3 .. name', None)]

## 상관관계 업데이트(Correlated Updates)
- 다른 테이블의 값을 사용해 테이블을 업데이트

In [61]:
# addresses 테이블에서 첫번째 email 주소를 선택
stmt = select([addresses.c.email_address]).\
            where(addresses.c.user_id == users.c.id).\
            limit(1)
conn.execute(stmt).fetchall()

2018-06-07 10:00:26,649 INFO sqlalchemy.engine.base.Engine SELECT addresses.email_address 
FROM addresses, users 
WHERE addresses.user_id = users.id
 LIMIT ? OFFSET ?
2018-06-07 10:00:26,650 INFO sqlalchemy.engine.base.Engine (1, 0)


[('jack@yahoo.com',)]

In [62]:
# 위에서 선택한 주소를 이용해 users 테이블의 fullname 컬럼을 update
conn.execute(users.update().values(fullname=stmt))
conn.execute(users.select()).fetchall()

2018-06-07 10:00:28,587 INFO sqlalchemy.engine.base.Engine UPDATE users SET fullname=(SELECT addresses.email_address 
FROM addresses 
WHERE addresses.user_id = users.id
 LIMIT ? OFFSET ?)
2018-06-07 10:00:28,588 INFO sqlalchemy.engine.base.Engine (1, 0)
2018-06-07 10:00:28,589 INFO sqlalchemy.engine.base.Engine COMMIT
2018-06-07 10:00:28,590 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users
2018-06-07 10:00:28,591 INFO sqlalchemy.engine.base.Engine ()


[(1, 'ed', 'jack@yahoo.com'),
 (2, 'mary', 'www@www.org'),
 (4, 'name1 .. name', None),
 (5, 'name2 .. name', None),
 (6, 'name3 .. name', None)]

## Parameter-Ordered Updates

- `update()`문의 SET 절에서 렌더링된 파라미터의 순서가 중요할 수 있다.
    - 예: `UPDATE some_table SET x = y + 10, y = 20`
    - 위 경우에는 `UPDATE some_table SET y = 20, x = y + 10` 이런 결과를 갖는다.
    - `preserve_parameter_order` 플래그를 사용할 수 있다.

In [ ]:
stmt = some_table.update(preserve_parameter_order=True).\
    values([(some_table.c.y, 20), (some_table.c.x, some_table.c.y + 10)])

## 삭제

- `delete()`
- `rowcount`: WHERE 절과 일치하는 행의 개수를 보여준다.

In [63]:
result = conn.execute(users.delete().where(users.c.name > 'm'))

2018-06-07 10:00:34,682 INFO sqlalchemy.engine.base.Engine DELETE FROM users WHERE users.name > ?
2018-06-07 10:00:34,683 INFO sqlalchemy.engine.base.Engine ('m',)
2018-06-07 10:00:34,685 INFO sqlalchemy.engine.base.Engine COMMIT


In [64]:
conn.execute(users.select()).fetchall()

2018-06-07 10:00:37,506 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users
2018-06-07 10:00:37,507 INFO sqlalchemy.engine.base.Engine ()


[(1, 'ed', 'jack@yahoo.com')]

In [65]:
# rowcount
result = conn.execute(users.delete())
result.rowcount

2018-06-07 10:00:38,345 INFO sqlalchemy.engine.base.Engine DELETE FROM users
2018-06-07 10:00:38,346 INFO sqlalchemy.engine.base.Engine ()
2018-06-07 10:00:38,347 INFO sqlalchemy.engine.base.Engine COMMIT


1